## Testing get_graph_sample_idx() function

This function returns a dictionary with a list of indexes in order to build ContinuousTimeGraphSample

For that, we do the same example showed in the paper ASYNCHRONOUS GRAPH GENERATORS in figure 2.

In [1]:
import numpy as np
from tqdm import trange

In [2]:
def get_graph_sample_idx(context_len, stride, idx_list, selection_size) -> dict:
    """
    This function takes all the Ids which has each sample in our mongodb database, and with its parameters creates a list 
    of indexes of each ContinuousTimeGraphSample
    :param context_len: input block length
    :param stride: number of steps the block is moved before it is considered the next input to the AGG 
    :param idx_list: all the indexes of samples located in mongodb database
    :param selection_size: it is a % in how many samples we want to remove
    :return: dict
    """

    # targets = set(random.sample(idx_list, int(selection_size * len(idx_list))))  # we select randomly the targets from idx_list
    targets = { 2,5,6,9,14,12}

    print('Total targets: ',len(targets))
    train_nodes = [k for k in idx_list if k not in targets] # O(1) targets is save it as by hash table
    N = len(train_nodes)
    print('Total train nodes: ', N)
    windows = []
    idx_indices = {idx: i for i, idx in enumerate(idx_list)} # we create a map between indexes and lists of each samples, it's take O(1) doing this 
    k=0
    graph_samples = dict()
    print('Creating indexes of nodes...')
    for i in trange(0, N, stride):
        max_idx = min(i + context_len, N) 
        window = train_nodes[i:max_idx] 
        if len(window)==context_len: # we ask to the window to have exact lenght of the block
            windows.append(window) 
            first_element_window = idx_indices[window[0]]
            last_element_window = idx_indices[window[-1]]
            for target in targets :  # we iterate over each target in order to see if it is in the range of the context block
                if first_element_window <= idx_indices[target] <= last_element_window :
                    graph_samples[k] = [window,target] 
                    k+=1
    return graph_samples # it returns a dict with a list of indexes

In [3]:
a = np.arange(0,17).tolist()
targets = { 2,5,6,9,14,12}
context_len = 3
stride = 2
get_graph_sample_idx(context_len = context_len, stride = stride, idx_list = a, selection_size = None)

Total targets:  6
Total train nodes:  11
Creating indexes of nodes...


100%|██████████| 6/6 [00:00<00:00, 51675.20it/s]

{0: [[0, 1, 3], 2],
 1: [[3, 4, 7], 5],
 2: [[3, 4, 7], 6],
 3: [[7, 8, 10], 9],
 4: [[10, 11, 13], 12],
 5: [[13, 15, 16], 14]}

The above output is the same to figure 2 in paper ASYNCHRONOUS GRAPH GENERATORS